<a href="https://colab.research.google.com/github/Karthikreddy1010/Deep-Learning_Basic_Projects/blob/main/Fashion_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# # CSIT 595 Assignment 1 — Generative Models on Fashion-MNIST
# **VAE · GAN · Diffusion Model (DDPM)**
#
# Run on Google Colab with GPU runtime.
# All output images and graphs are saved to ./outputs/

# ============================================================
# 0. Imports & Setup
# ============================================================

import os, time, math, copy
import numpy as np
import matplotlib
matplotlib.use('Agg')          # Non-interactive backend — saves files without display
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms
from scipy.linalg import sqrtm
import pandas as pd

# ── Output directory
OUTPUT_DIR = './outputs'
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f'All images/graphs will be saved to: {os.path.abspath(OUTPUT_DIR)}')

# ── GPU setup
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {DEVICE}')
if DEVICE.type == 'cuda':
    print(f'  GPU: {torch.cuda.get_device_name(0)}')
    print(f'  VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')
    # Speed-up flags
    torch.backends.cudnn.benchmark = True
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

torch.manual_seed(42)
np.random.seed(42)

# ── Hyperparameters
BATCH_SIZE   = 128
IMG_SIZE     = 32
NC           = 1
LATENT_DIM   = 128
VAE_EPOCHS   = 30
GAN_EPOCHS   = 50
DIFF_STEPS   = 500
DIFF_EPOCHS  = 30
EVAL_SAMPLES = 5000
NUM_CLASSES  = 10

CLASS_NAMES = ['T-shirt','Trouser','Pullover','Dress','Coat',
               'Sandal','Shirt','Sneaker','Bag','Ankle boot']
print('Setup complete.')


# ============================================================
# Helper: save figure util
# ============================================================

def savefig(filename, dpi=150):
    """Save current matplotlib figure to OUTPUT_DIR and close it."""
    path = os.path.join(OUTPUT_DIR, filename)
    plt.savefig(path, dpi=dpi, bbox_inches='tight')
    plt.close()
    print(f'  [saved] {path}')


# ============================================================
# 1. Common Setup & Evaluator Network
# ============================================================

# ── DataLoaders
num_workers = 4 if DEVICE.type == 'cuda' else 2
transform = transforms.Compose([
    transforms.Pad(2),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

train_ds = datasets.FashionMNIST('./data', train=True,  download=True, transform=transform)
test_ds  = datasets.FashionMNIST('./data', train=False, download=True, transform=transform)

train_loader = DataLoader(
    train_ds, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=num_workers, pin_memory=(DEVICE.type == 'cuda'),
    persistent_workers=True
)
test_loader = DataLoader(
    test_ds, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=num_workers, pin_memory=(DEVICE.type == 'cuda'),
    persistent_workers=True
)
print(f'Train: {len(train_ds):,}  |  Test: {len(test_ds):,}')


# ── Evaluator CNN
class EvaluatorCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1,  32, 3, padding=1), nn.BatchNorm2d(32),  nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64),  nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.Conv2d(128,256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(256,256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.AdaptiveAvgPool2d(4),
        )
        self.penultimate = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256*4*4, 512), nn.ReLU(), nn.Dropout(0.4),
        )
        self.classifier = nn.Linear(512, num_classes)

    def forward(self, x, return_features=False):
        feat = self.features(x)
        feat = self.penultimate(feat)
        if return_features:
            return feat
        return self.classifier(feat)


def train_evaluator(model, train_loader, test_loader, epochs=15):
    opt   = torch.optim.Adam(model.parameters(), lr=1e-3)
    sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, epochs)
    scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE.type == 'cuda'))

    for ep in range(1, epochs+1):
        model.train()
        total_loss = correct = total = 0
        for x, y in train_loader:
            x, y = x.to(DEVICE, non_blocking=True), y.to(DEVICE, non_blocking=True)
            opt.zero_grad()
            with torch.cuda.amp.autocast(enabled=(DEVICE.type == 'cuda')):
                logits = model(x)
                loss   = F.cross_entropy(logits, y)
            scaler.scale(loss).backward()
            scaler.step(opt)
            scaler.update()
            total_loss += loss.item() * x.size(0)
            correct    += (logits.argmax(1) == y).sum().item()
            total      += x.size(0)
        sched.step()
        if ep % 5 == 0 or ep == 1:
            print(f'  Epoch {ep:3d}  loss={total_loss/total:.4f}  acc={correct/total*100:.1f}%')

    model.eval()
    correct = total = 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(DEVICE, non_blocking=True), y.to(DEVICE, non_blocking=True)
            correct += (model(x).argmax(1) == y).sum().item()
            total   += x.size(0)
    print(f'\n✓ Evaluator Test Accuracy: {correct/total*100:.2f}%')
    return model


print('Training Evaluator CNN...')
evaluator = EvaluatorCNN().to(DEVICE)
evaluator = train_evaluator(evaluator, train_loader, test_loader, epochs=15)
torch.save(evaluator.state_dict(), 'evaluator.pth')


# ── Evaluation utilities

def get_features_and_labels(model, images_tensor, batch_size=256):
    model.eval()
    feats, preds = [], []
    with torch.no_grad():
        for i in range(0, len(images_tensor), batch_size):
            xb = images_tensor[i:i+batch_size].to(DEVICE, non_blocking=True)
            feats.append(model(xb, return_features=True).cpu())
            preds.append(model(xb).argmax(1).cpu())
    return torch.cat(feats).numpy(), torch.cat(preds).numpy()


def compute_cfid(real_feats, gen_feats):
    mu1, sig1 = real_feats.mean(0), np.cov(real_feats, rowvar=False)
    mu2, sig2 = gen_feats.mean(0),  np.cov(gen_feats,  rowvar=False)
    diff    = mu1 - mu2
    covmean = sqrtm(sig1 @ sig2)
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    return float(diff @ diff + np.trace(sig1 + sig2 - 2*covmean))


def coverage_metrics(pred_labels, real_label_dist=None):
    counts = np.bincount(pred_labels, minlength=NUM_CLASSES).astype(float)
    p      = counts / counts.sum()
    entropy = -np.sum(p * np.log(p + 1e-8))
    kl = None
    if real_label_dist is not None:
        kl = np.sum(real_label_dist * np.log((real_label_dist+1e-8)/(p+1e-8)))
    return p, entropy, kl


def measure_sampling_speed(sample_fn, n=EVAL_SAMPLES, batch=256):
    all_imgs, remaining = [], n
    t0 = time.time()
    while remaining > 0:
        bs = min(batch, remaining)
        all_imgs.append(sample_fn(bs).cpu())
        remaining -= bs
    elapsed = time.time() - t0
    speed = n / elapsed
    print(f'  {n} images  →  {speed:.1f} imgs/s  ({elapsed:.1f}s total)')
    return torch.cat(all_imgs), elapsed, speed


def save_grid(imgs, title='', filename='grid.png', nrow=10, figsize=(12, 12)):
    """Save an image grid to file."""
    imgs = (imgs.clamp(-1, 1) + 1) / 2
    grid = torchvision.utils.make_grid(imgs[:nrow*nrow], nrow=nrow, padding=2)
    plt.figure(figsize=figsize)
    plt.imshow(grid.permute(1, 2, 0).squeeze(), cmap='gray')
    plt.title(title, fontsize=13)
    plt.axis('off')
    plt.tight_layout()
    savefig(filename)


def save_label_hist(dist, title='Label Distribution', filename='label_hist.png'):
    plt.figure(figsize=(8, 3))
    plt.bar(range(NUM_CLASSES), dist)
    plt.xticks(range(NUM_CLASSES), CLASS_NAMES, rotation=45, ha='right')
    plt.ylabel('Proportion')
    plt.title(title)
    plt.tight_layout()
    savefig(filename)


# Pre-compute real features
print('Extracting real features for cFID baseline...')
real_imgs_all = torch.stack([train_ds[i][0] for i in range(EVAL_SAMPLES)])
real_feats, real_labels = get_features_and_labels(evaluator, real_imgs_all)
REAL_LABEL_DIST = np.bincount(real_labels, minlength=NUM_CLASSES).astype(float)
REAL_LABEL_DIST /= REAL_LABEL_DIST.sum()
print('Done.')


# ============================================================
# 2. Variational Autoencoder (VAE)
# ============================================================

class VAEEncoder(nn.Module):
    def __init__(self, latent_dim=LATENT_DIM):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1,   32, 4, 2, 1), nn.LeakyReLU(0.2),
            nn.Conv2d(32,  64, 4, 2, 1), nn.BatchNorm2d(64),  nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 4, 2, 1), nn.BatchNorm2d(128), nn.LeakyReLU(0.2),
            nn.Conv2d(128,256, 4, 2, 1), nn.BatchNorm2d(256), nn.LeakyReLU(0.2),
            nn.Flatten(),
            nn.Linear(256*2*2, 512), nn.ReLU(),
        )
        self.fc_mu  = nn.Linear(512, latent_dim)
        self.fc_log = nn.Linear(512, latent_dim)

    def forward(self, x):
        h = self.net(x)
        return self.fc_mu(h), self.fc_log(h)


class VAEDecoder(nn.Module):
    def __init__(self, latent_dim=LATENT_DIM):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(latent_dim, 512), nn.ReLU(),
            nn.Linear(512, 256*2*2),    nn.ReLU()
        )
        self.net = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 4, 2, 1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.ConvTranspose2d(128,  64, 4, 2, 1), nn.BatchNorm2d(64),  nn.ReLU(),
            nn.ConvTranspose2d( 64,  32, 4, 2, 1), nn.BatchNorm2d(32),  nn.ReLU(),
            nn.ConvTranspose2d( 32,   1, 4, 2, 1), nn.Tanh(),
        )

    def forward(self, z):
        return self.net(self.fc(z).view(-1, 256, 2, 2))


class VAE(nn.Module):
    def __init__(self, latent_dim=LATENT_DIM):
        super().__init__()
        self.encoder = VAEEncoder(latent_dim)
        self.decoder = VAEDecoder(latent_dim)

    def reparameterise(self, mu, log_var):
        return mu + (0.5 * log_var).exp() * torch.randn_like(mu)

    def forward(self, x):
        mu, log_var = self.encoder(x)
        return self.decoder(self.reparameterise(mu, log_var)), mu, log_var

    @torch.no_grad()
    def sample(self, n):
        return self.decoder(torch.randn(n, LATENT_DIM, device=DEVICE))


def vae_loss(recon, x, mu, log_var, beta=1.0):
    r = F.mse_loss(recon, x, reduction='sum') / x.size(0)
    k = -0.5 * (1 + log_var - mu.pow(2) - log_var.exp()).sum(1).mean()
    return r + beta*k, r, k


print(f'VAE parameters: {sum(p.numel() for p in VAE().parameters()):,}')

vae = VAE().to(DEVICE)
opt_vae    = torch.optim.Adam(vae.parameters(), lr=2e-4)
sched_vae  = torch.optim.lr_scheduler.CosineAnnealingLR(opt_vae, VAE_EPOCHS)
scaler_vae = torch.cuda.amp.GradScaler(enabled=(DEVICE.type == 'cuda'))

hist_vae = {'total': [], 'recon': [], 'kl': []}
t0_vae   = time.time()

for ep in range(1, VAE_EPOCHS+1):
    vae.train()
    ep_t = ep_r = ep_k = n = 0
    for x, _ in train_loader:
        x = x.to(DEVICE, non_blocking=True)
        opt_vae.zero_grad()
        with torch.cuda.amp.autocast(enabled=(DEVICE.type == 'cuda')):
            recon, mu, lv = vae(x)
            loss, r, k    = vae_loss(recon, x, mu, lv)
        scaler_vae.scale(loss).backward()
        scaler_vae.step(opt_vae)
        scaler_vae.update()
        ep_t += loss.item(); ep_r += r.item(); ep_k += k.item(); n += 1
    sched_vae.step()
    hist_vae['total'].append(ep_t/n)
    hist_vae['recon'].append(ep_r/n)
    hist_vae['kl'].append(ep_k/n)
    if ep % 5 == 0 or ep == 1:
        print(f'VAE ep{ep:3d}  total={ep_t/n:.3f}  recon={ep_r/n:.3f}  kl={ep_k/n:.3f}')

vae_train_time = time.time() - t0_vae
print(f'VAE training time: {vae_train_time/60:.1f} min')
torch.save(vae.state_dict(), 'vae.pth')

# ── Training curves
fig, axes = plt.subplots(1, 3, figsize=(14, 4))
for ax, key, c in zip(axes, ['total','recon','kl'], ['tab:blue','tab:orange','tab:green']):
    ax.plot(hist_vae[key], color=c)
    ax.set_title(f'VAE {key.capitalize()} Loss')
    ax.set_xlabel('Epoch'); ax.set_ylabel('Loss')
plt.suptitle('VAE Training Curves', fontsize=14)
plt.tight_layout()
savefig('vae_training_curves.png')

# ── Reconstructions
vae.eval()
with torch.no_grad():
    real_batch          = next(iter(test_loader))[0][:50].to(DEVICE)
    recon_batch, _, _   = vae(real_batch)

interleaved = torch.stack([t for pair in zip(real_batch.cpu(), recon_batch.cpu()) for t in pair])
save_grid(interleaved, 'VAE Reconstructions (original | reconstruction pairs)',
          'vae_reconstructions.png', nrow=10)

# ── Unconditional samples
samples_vae = vae.sample(100).cpu()
save_grid(samples_vae, 'VAE — 100 Unconditional Samples', 'vae_samples.png', nrow=10)

# ── Latent interpolation
with torch.no_grad():
    x1 = test_ds[0][0].unsqueeze(0).to(DEVICE)
    x2 = test_ds[1][0].unsqueeze(0).to(DEVICE)
    mu1, _ = vae.encoder(x1)
    mu2, _ = vae.encoder(x2)
    alphas  = torch.linspace(0, 1, 10).to(DEVICE)
    interp_vae = torch.cat([vae.decoder(mu1*(1-a)+mu2*a) for a in alphas]).cpu()
save_grid(interp_vae, 'VAE Latent Interpolation (10 steps)', 'vae_interpolation.png', nrow=10)

# ── Evaluate VAE
print('Evaluating VAE...')
vae_imgs, vae_elapsed, vae_speed = measure_sampling_speed(vae.sample)
vae_gen_feats, vae_gen_labels    = get_features_and_labels(evaluator, vae_imgs)
vae_cfid                          = compute_cfid(real_feats, vae_gen_feats)
vae_dist, vae_ent, vae_kl        = coverage_metrics(vae_gen_labels, REAL_LABEL_DIST)

print(f'VAE cFID:           {vae_cfid:.2f}')
print(f'VAE Label Entropy:  {vae_ent:.3f}  (max={math.log(10):.3f})')
print(f'VAE KL Divergence:  {vae_kl:.4f}')
print(f'VAE Sampling speed: {vae_speed:.1f} imgs/s  |  5k imgs in {vae_elapsed:.1f}s')
save_label_hist(vae_dist, 'VAE — Generated Label Distribution', 'vae_label_dist.png')


# ============================================================
# 3. Generative Adversarial Network (DCGAN)
# ============================================================

def weights_init(m):
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d)):
        nn.init.normal_(m.weight, 0.0, 0.02)
    if isinstance(m, nn.BatchNorm2d):
        nn.init.normal_(m.weight, 1.0, 0.02)
        nn.init.constant_(m.bias, 0)


class Generator(nn.Module):
    def __init__(self, z_dim=LATENT_DIM):
        super().__init__()
        self.net = nn.Sequential(
            nn.ConvTranspose2d(z_dim, 512, 2, 1, 0, bias=False), nn.BatchNorm2d(512), nn.ReLU(True),
            nn.ConvTranspose2d(512,  256, 4, 2, 1, bias=False), nn.BatchNorm2d(256), nn.ReLU(True),
            nn.ConvTranspose2d(256,  128, 4, 2, 1, bias=False), nn.BatchNorm2d(128), nn.ReLU(True),
            nn.ConvTranspose2d(128,   64, 4, 2, 1, bias=False), nn.BatchNorm2d(64),  nn.ReLU(True),
            nn.ConvTranspose2d( 64,    1, 4, 2, 1, bias=False), nn.Tanh(),
        )
    def forward(self, z):
        return self.net(z.view(-1, LATENT_DIM, 1, 1))

    @torch.no_grad()
    def sample(self, n):
        return self(torch.randn(n, LATENT_DIM, device=DEVICE))


class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(  1,  64, 4, 2, 1, bias=False), nn.LeakyReLU(0.2, True),
            nn.Conv2d( 64, 128, 4, 2, 1, bias=False), nn.BatchNorm2d(128), nn.LeakyReLU(0.2, True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False), nn.BatchNorm2d(256), nn.LeakyReLU(0.2, True),
            nn.Conv2d(256, 512, 4, 2, 1, bias=False), nn.BatchNorm2d(512), nn.LeakyReLU(0.2, True),
            nn.Conv2d(512,   1, 2, 1, 0, bias=False), nn.Flatten(),
            # No Sigmoid here — BCEWithLogitsLoss applies it internally (autocast-safe)
        )
    def forward(self, x):
        return self.net(x)


G = Generator().to(DEVICE).apply(weights_init)
D = Discriminator().to(DEVICE).apply(weights_init)
print(f'Generator params:     {sum(p.numel() for p in G.parameters()):,}')
print(f'Discriminator params: {sum(p.numel() for p in D.parameters()):,}')

opt_G     = torch.optim.Adam(G.parameters(), lr=2e-4, betas=(0.5, 0.999))
opt_D     = torch.optim.Adam(D.parameters(), lr=2e-4, betas=(0.5, 0.999))
criterion = nn.BCEWithLogitsLoss()   # autocast-safe; no Sigmoid needed in D
scaler_G  = torch.cuda.amp.GradScaler(enabled=(DEVICE.type == 'cuda'))
scaler_D  = torch.cuda.amp.GradScaler(enabled=(DEVICE.type == 'cuda'))

fixed_noise   = torch.randn(100, LATENT_DIM, device=DEVICE)
hist_gan      = {'G': [], 'D': []}
gan_checkpoints = {}
CKPT_EPOCHS   = [1, 10, 25, GAN_EPOCHS]

t0_gan = time.time()

for ep in range(1, GAN_EPOCHS+1):
    G.train(); D.train()
    ep_G = ep_D = n = 0
    for x, _ in train_loader:
        bs   = x.size(0)
        real = x.to(DEVICE, non_blocking=True)
        rl   = torch.full((bs, 1), 0.9, device=DEVICE)
        fl   = torch.zeros(bs, 1, device=DEVICE)

        # ── Train D
        opt_D.zero_grad()
        with torch.cuda.amp.autocast(enabled=(DEVICE.type == 'cuda')):
            z     = torch.randn(bs, LATENT_DIM, device=DEVICE)
            lossD = criterion(D(real), rl) + criterion(D(G(z).detach()), fl)
        scaler_D.scale(lossD).backward()
        scaler_D.step(opt_D); scaler_D.update()

        # ── Train G
        opt_G.zero_grad()
        with torch.cuda.amp.autocast(enabled=(DEVICE.type == 'cuda')):
            z     = torch.randn(bs, LATENT_DIM, device=DEVICE)
            lossG = criterion(D(G(z)), torch.ones(bs, 1, device=DEVICE))
        scaler_G.scale(lossG).backward()
        scaler_G.step(opt_G); scaler_G.update()

        ep_G += lossG.item(); ep_D += lossD.item(); n += 1

    hist_gan['G'].append(ep_G/n)
    hist_gan['D'].append(ep_D/n)

    if ep in CKPT_EPOCHS:
        G.eval()
        with torch.no_grad():
            ck = torch.cat([G(torch.randn(256, LATENT_DIM, device=DEVICE)).cpu() for _ in range(20)])
        _, ck_lbl = get_features_and_labels(evaluator, ck)
        d = np.bincount(ck_lbl, minlength=NUM_CLASSES).astype(float)
        gan_checkpoints[ep] = d / d.sum()
        print(f'GAN ep{ep:3d}  G={ep_G/n:.4f}  D={ep_D/n:.4f}  [checkpoint]')
    elif ep % 10 == 0:
        print(f'GAN ep{ep:3d}  G={ep_G/n:.4f}  D={ep_D/n:.4f}')

gan_train_time = time.time() - t0_gan
print(f'GAN training time: {gan_train_time/60:.1f} min')
torch.save(G.state_dict(), 'generator.pth')

# ── Training curves
plt.figure(figsize=(8, 4))
plt.plot(hist_gan['G'], label='Generator')
plt.plot(hist_gan['D'], label='Discriminator')
plt.xlabel('Epoch'); plt.ylabel('Loss')
plt.title('GAN Training Curves'); plt.legend()
plt.tight_layout()
savefig('gan_training_curves.png')

# ── Unconditional samples
G.eval()
with torch.no_grad():
    gan_samples = G(fixed_noise).cpu()
save_grid(gan_samples, 'GAN — 100 Unconditional Samples', 'gan_samples.png', nrow=10)

# ── Latent interpolation
with torch.no_grad():
    z1 = torch.randn(1, LATENT_DIM, device=DEVICE)
    z2 = torch.randn(1, LATENT_DIM, device=DEVICE)
    alphas     = torch.linspace(0, 1, 10).to(DEVICE)
    interp_gan = torch.cat([G(z1*(1-a)+z2*a) for a in alphas]).cpu()
save_grid(interp_gan, 'GAN Latent Interpolation (10 steps)', 'gan_interpolation.png', nrow=10)

# ── Mode collapse watch
fig, axes = plt.subplots(1, len(gan_checkpoints), figsize=(14, 3))
for ax, (ep, dist) in zip(axes, gan_checkpoints.items()):
    ax.bar(range(NUM_CLASSES), dist)
    ax.set_title(f'Epoch {ep}')
    ax.set_xticks(range(NUM_CLASSES))
    ax.set_xticklabels([c[:4] for c in CLASS_NAMES], rotation=45, ha='right', fontsize=7)
plt.suptitle('GAN Mode Collapse Watch — Label Histograms at Checkpoints', fontsize=12)
plt.tight_layout()
savefig('gan_mode_collapse_watch.png')

# ── Evaluate GAN
print('Evaluating GAN...')
gan_imgs, gan_elapsed, gan_speed = measure_sampling_speed(G.sample)
gan_gen_feats, gan_gen_labels    = get_features_and_labels(evaluator, gan_imgs)
gan_cfid                          = compute_cfid(real_feats, gan_gen_feats)
gan_dist, gan_ent, gan_kl        = coverage_metrics(gan_gen_labels, REAL_LABEL_DIST)

print(f'GAN cFID:           {gan_cfid:.2f}')
print(f'GAN Label Entropy:  {gan_ent:.3f}')
print(f'GAN KL Divergence:  {gan_kl:.4f}')
print(f'GAN Sampling speed: {gan_speed:.1f} imgs/s  |  5k imgs in {gan_elapsed:.1f}s')
save_label_hist(gan_dist, 'GAN — Generated Label Distribution', 'gan_label_dist.png')


# ============================================================
# 4. Diffusion Model (DDPM-style U-Net)
# ============================================================

# ── Noise schedule
T         = DIFF_STEPS
betas     = torch.linspace(1e-4, 0.02, T).to(DEVICE)
alphas    = 1.0 - betas
alpha_bar = torch.cumprod(alphas, dim=0)
alpha_bar_prev = F.pad(alpha_bar[:-1], (1, 0), value=1.0)


def q_sample(x0, t, noise=None):
    if noise is None:
        noise = torch.randn_like(x0)
    ab = alpha_bar[t].view(-1, 1, 1, 1)
    return ab.sqrt()*x0 + (1-ab).sqrt()*noise, noise


# ── U-Net with sinusoidal time embeddings

class SinusoidalEmb(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim
    def forward(self, t):
        half  = self.dim // 2
        freqs = torch.exp(-math.log(10000) * torch.arange(half, device=t.device) / half)
        emb   = t.float()[:, None] * freqs[None]
        return torch.cat([emb.sin(), emb.cos()], dim=-1)


class ResBlock(nn.Module):
    def __init__(self, ch, t_dim=128):
        super().__init__()
        self.norm1  = nn.GroupNorm(8, ch)
        self.conv1  = nn.Conv2d(ch, ch, 3, padding=1)
        self.t_proj = nn.Linear(t_dim, ch)
        self.norm2  = nn.GroupNorm(8, ch)
        self.conv2  = nn.Conv2d(ch, ch, 3, padding=1)
    def forward(self, x, te):
        h = self.conv1(F.silu(self.norm1(x)))
        h = h + self.t_proj(F.silu(te))[:, :, None, None]
        return self.conv2(F.silu(self.norm2(h))) + x


class UNet(nn.Module):
    def __init__(self, ch=64, t_dim=128):
        super().__init__()
        self.t_emb = nn.Sequential(
            SinusoidalEmb(t_dim),
            nn.Linear(t_dim, t_dim), nn.SiLU(),
            nn.Linear(t_dim, t_dim))
        self.enc0  = nn.Conv2d(1, ch, 3, padding=1)
        self.enc1  = ResBlock(ch, t_dim)
        self.down1 = nn.Conv2d(ch, ch*2, 4, 2, 1)
        self.enc2  = ResBlock(ch*2, t_dim)
        self.down2 = nn.Conv2d(ch*2, ch*4, 4, 2, 1)
        self.bot1  = ResBlock(ch*4, t_dim)
        self.bot2  = ResBlock(ch*4, t_dim)
        self.up1   = nn.ConvTranspose2d(ch*4, ch*2, 4, 2, 1)
        self.dec1  = ResBlock(ch*4, t_dim)
        self.up2   = nn.ConvTranspose2d(ch*4, ch, 4, 2, 1)
        self.dec2  = ResBlock(ch*2, t_dim)
        self.out   = nn.Sequential(
            nn.GroupNorm(8, ch*2), nn.SiLU(),
            nn.Conv2d(ch*2, 1, 3, padding=1))
    def forward(self, x, t):
        te = self.t_emb(t)
        e0 = self.enc0(x)
        e1 = self.enc1(e0, te)
        e2 = self.enc2(self.down1(e1), te)
        b  = self.bot2(self.bot1(self.down2(e2), te), te)
        d1 = self.dec1(torch.cat([self.up1(b),  e2], 1), te)
        d2 = self.dec2(torch.cat([self.up2(d1), e1], 1), te)
        return self.out(d2)


unet = UNet().to(DEVICE)
print(f'U-Net parameters: {sum(p.numel() for p in unet.parameters()):,}')

opt_diff    = torch.optim.Adam(unet.parameters(), lr=2e-4)
sched_diff  = torch.optim.lr_scheduler.CosineAnnealingLR(opt_diff, DIFF_EPOCHS)
scaler_diff = torch.cuda.amp.GradScaler(enabled=(DEVICE.type == 'cuda'))

hist_diff = []
t0_diff   = time.time()

for ep in range(1, DIFF_EPOCHS+1):
    unet.train()
    ep_loss = n = 0
    for x, _ in train_loader:
        x      = x.to(DEVICE, non_blocking=True)
        t_rand = torch.randint(0, T, (x.size(0),), device=DEVICE)
        opt_diff.zero_grad()
        with torch.cuda.amp.autocast(enabled=(DEVICE.type == 'cuda')):
            xt, noise = q_sample(x, t_rand)
            loss      = F.mse_loss(unet(xt, t_rand), noise)
        scaler_diff.scale(loss).backward()
        scaler_diff.step(opt_diff); scaler_diff.update()
        ep_loss += loss.item(); n += 1
    sched_diff.step()
    hist_diff.append(ep_loss/n)
    if ep % 5 == 0 or ep == 1:
        print(f'DDPM ep{ep:3d}  MSE={ep_loss/n:.5f}')

diff_train_time = time.time() - t0_diff
print(f'Diffusion training time: {diff_train_time/60:.1f} min')
torch.save(unet.state_dict(), 'unet.pth')

# ── Training curve
plt.figure(figsize=(8, 4))
plt.plot(hist_diff)
plt.xlabel('Epoch'); plt.ylabel('MSE')
plt.title('Diffusion — Noise Prediction MSE')
plt.tight_layout()
savefig('diff_training_curve.png')


# ── DDPM reverse sampling
@torch.no_grad()
def ddpm_sample(model, n, steps=T):
    model.eval()
    t_seq = torch.linspace(T-1, 0, steps, dtype=torch.long, device=DEVICE)
    x     = torch.randn(n, 1, IMG_SIZE, IMG_SIZE, device=DEVICE)
    for t_idx in t_seq:
        tb       = t_idx.repeat(n)
        eps_pred = model(x, tb)
        ab_t     = alpha_bar[t_idx]
        ab_tp    = alpha_bar_prev[t_idx]
        beta_t   = betas[t_idx]
        x0_pred  = ((x - (1-ab_t).sqrt()*eps_pred) / ab_t.sqrt()).clamp(-1, 1)
        mean     = (ab_tp.sqrt()*beta_t/(1-ab_t))*x0_pred + \
                   (alphas[t_idx].sqrt()*(1-ab_tp)/(1-ab_t))*x
        if t_idx > 0:
            x = mean + (beta_t*(1-ab_tp)/(1-ab_t)).sqrt() * torch.randn_like(x)
        else:
            x = mean
    return x


# ── Sample at multiple step counts
diff_results = {}
for steps in [500, 200, 100, 50, 20]:
    print(f'\nSampling {steps} steps...')
    t0   = time.time()
    imgs = ddpm_sample(unet, 100, steps=steps).cpu()
    spd  = 100 / (time.time() - t0)
    save_grid(imgs, f'Diffusion — 100 samples @ {steps} steps  ({spd:.1f} imgs/s)',
              f'diff_samples_{steps}steps.png', nrow=10)
    diff_results[steps] = {'imgs': imgs, 'speed': spd}

# ── Full evaluation at each step count
diff_metrics = {}
for steps in diff_results:
    print(f'\n--- Diffusion @{steps} steps ---')
    fn = lambda n, s=steps: ddpm_sample(unet, n, steps=s)
    gen_imgs, elapsed, spd = measure_sampling_speed(fn)
    feats, labels          = get_features_and_labels(evaluator, gen_imgs)
    cfid                   = compute_cfid(real_feats, feats)
    dist, ent, kl          = coverage_metrics(labels, REAL_LABEL_DIST)
    diff_metrics[steps]    = {'cfid': cfid, 'entropy': ent, 'kl': kl,
                               'speed': spd, 'elapsed': elapsed, 'dist': dist}
    print(f'  cFID={cfid:.2f}  entropy={ent:.3f}  KL={kl:.4f}  speed={spd:.1f} imgs/s')
    save_label_hist(dist, f'Diffusion @{steps} steps — Label Distribution',
                    f'diff_label_dist_{steps}steps.png')


# ============================================================
# 5. Performance Comparison & Trilemma Analysis
# ============================================================

vae_params  = sum(p.numel() for p in vae.parameters())
gan_params  = sum(p.numel() for p in G.parameters()) + sum(p.numel() for p in D.parameters())
diff_params = sum(p.numel() for p in unet.parameters())

rows = [
    {'Model': 'VAE',
     'cFID': round(vae_cfid, 2), 'Entropy': round(vae_ent, 3),
     'KL Div': round(vae_kl, 4), 'Imgs/sec': round(vae_speed, 1),
     'Time 5k (s)': round(vae_elapsed, 1), '# Params': f'{vae_params:,}',
     'Train time (min)': round(vae_train_time/60, 1)},
    {'Model': 'GAN',
     'cFID': round(gan_cfid, 2), 'Entropy': round(gan_ent, 3),
     'KL Div': round(gan_kl, 4), 'Imgs/sec': round(gan_speed, 1),
     'Time 5k (s)': round(gan_elapsed, 1), '# Params': f'{gan_params:,}',
     'Train time (min)': round(gan_train_time/60, 1)},
]
for steps, m in diff_metrics.items():
    rows.append({
        'Model': f'Diffusion ({steps}steps)',
        'cFID': round(m['cfid'], 2), 'Entropy': round(m['entropy'], 3),
        'KL Div': round(m['kl'], 4), 'Imgs/sec': round(m['speed'], 1),
        'Time 5k (s)': round(m['elapsed'], 1), '# Params': f'{diff_params:,}',
        'Train time (min)': round(diff_train_time/60, 1)
    })

df = pd.DataFrame(rows)
print('\n' + df.to_string(index=False))

# Save summary table as CSV
csv_path = os.path.join(OUTPUT_DIR, 'results_summary.csv')
df.to_csv(csv_path, index=False)
print(f'  [saved] {csv_path}')

# ── Bar charts for all three trilemma axes
models    = ['VAE', 'GAN'] + [f'Diff-{s}' for s in diff_metrics]
cfids     = [vae_cfid, gan_cfid]   + [m['cfid']    for m in diff_metrics.values()]
entropies = [vae_ent,  gan_ent]    + [m['entropy'] for m in diff_metrics.values()]
speeds    = [vae_speed, gan_speed] + [m['speed']   for m in diff_metrics.values()]

fig, axes = plt.subplots(1, 3, figsize=(16, 4))
for ax, vals, title, c in zip(axes,
        [cfids, entropies, speeds],
        ['cFID ↓', 'Entropy ↑', 'Sampling Speed ↑'],
        ['salmon', 'steelblue', 'seagreen']):
    ax.bar(models, vals, color=c)
    ax.set_title(title, fontsize=12)
    ax.set_xticklabels(models, rotation=30, ha='right', fontsize=8)
plt.suptitle('Generative Trilemma Comparison', fontsize=14)
plt.tight_layout()
savefig('trilemma_comparison.png')

# ── Diffusion speed-quality tradeoff
sc  = list(diff_metrics.keys())
fig, ax1 = plt.subplots(figsize=(8, 4))
ax2 = ax1.twinx()
ax1.plot(sc, [diff_metrics[s]['cfid']  for s in sc], 'o-', color='tomato',     label='cFID')
ax2.plot(sc, [diff_metrics[s]['speed'] for s in sc], 's-', color='dodgerblue', label='Speed')
ax1.set_xlabel('Diffusion Steps')
ax1.set_ylabel('cFID ↓', color='tomato')
ax2.set_ylabel('Imgs/sec ↑', color='dodgerblue')
ax1.invert_xaxis()
plt.title('Diffusion: Speed–Quality Tradeoff')
fig.legend(loc='upper right', bbox_to_anchor=(0.85, 0.88))
plt.tight_layout()
savefig('diff_speed_quality_tradeoff.png')

# ── Print final summary of saved files
print('\n' + '='*60)
print('ALL SAVED FILES IN', os.path.abspath(OUTPUT_DIR))
print('='*60)
for f in sorted(os.listdir(OUTPUT_DIR)):
    fpath = os.path.join(OUTPUT_DIR, f)
    size  = os.path.getsize(fpath) / 1024
    print(f'  {f:45s}  {size:7.1f} KB')

print("""
=== TRILEMMA ANALYSIS ===

1. SAMPLE QUALITY (cFID ↓)
   GAN usually achieves the sharpest samples and lowest cFID due to adversarial training.
   Diffusion at full steps is competitive but slower to generate.
   VAE produces blurrier samples (pixel-MSE objective), giving a higher cFID.

2. DIVERSITY / COVERAGE (Entropy ↑, KL ↓)
   VAE has the best coverage — it reconstructs across the full training distribution.
   GAN may collapse to a subset of modes, especially in early training (visible in histograms).
   Diffusion achieves good coverage when given sufficient denoising steps.

3. SAMPLING SPEED (imgs/sec ↑)
   GAN and VAE are single forward-pass models — orders of magnitude faster than diffusion.
   Diffusion with 20–50 steps approaches a practical speed while keeping quality reasonable.

FAILURE MODES
   VAE  : Blur from Gaussian decoder / pixel-MSE loss.
   GAN  : Training instability; early mode collapse visible in checkpoint label histograms.
   Diff : Slow; quality noticeably degrades below ~50 steps.

SUMMARY — The Generative Trilemma
   No single model dominates all three axes simultaneously:
   • VAE   → fastest + most diverse, but lowest quality.
   • GAN   → fastest + sharpest,     but diversity risk.
   • Diff  → best quality + coverage, but slowest without step reduction.
""")

All images/graphs will be saved to: /content/outputs
Device: cuda
  GPU: Tesla T4
  VRAM: 15.6 GB
Setup complete.


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:424: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  self.check_worker_number_rationality()


Train: 60,000  |  Test: 10,000
Training Evaluator CNN...


/tmp/ipython-input-1398/3455737240.py:132: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE.type == 'cuda'))
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:432: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  self.check_worker_number_rationality()
/tmp/ipython-input-1398/3455737240.py:140: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE.type == 'cuda')):


  Epoch   1  loss=0.3902  acc=85.7%
  Epoch   5  loss=0.1570  acc=94.2%
  Epoch  10  loss=0.0401  acc=98.5%
  Epoch  15  loss=0.0035  acc=99.9%

✓ Evaluator Test Accuracy: 94.31%
Extracting real features for cFID baseline...
Done.
VAE parameters: 2,626,817


...

  cFID=3039.89  entropy=0.550  KL=7.8337  speed=31.9 imgs/s
  [saved] ./outputs/diff_label_dist_50steps.png

--- Diffusion @20 steps ---
  5000 images  →  80.0 imgs/s  (62.5s total)


/tmp/ipython-input-1398/3455737240.py:187: LinAlgWarning: Matrix is singular. The result might be inaccurate or the array might not have a square root.
  covmean = sqrtm(sig1 @ sig2)


  cFID=3026.39  entropy=0.558  KL=7.7136  speed=80.0 imgs/s
  [saved] ./outputs/diff_label_dist_20steps.png

               Model    cFID  Entropy  KL Div  Imgs/sec  Time 5k (s)  # Params  Train time (min)
                 VAE  658.44    2.275  0.0302   28577.4          0.2 2,626,817               8.5
                 GAN   94.07    2.287  0.0153   11226.9          0.4 5,774,976              15.9
Diffusion (500steps)  128.30    2.282  0.0275       3.2       1561.2 5,826,817              25.0
Diffusion (200steps) 3128.08    0.741  6.6262       8.0        622.5 5,826,817              25.0
Diffusion (100steps) 3073.25    0.561  7.8581      16.0        312.1 5,826,817              25.0
 Diffusion (50steps) 3039.89    0.550  7.8337      31.9        156.7 5,826,817              25.0
 Diffusion (20steps) 3026.39    0.558  7.7136      80.0         62.5 5,826,817              25.0
  [saved] ./outputs/results_summary.csv


/tmp/ipython-input-1398/3455737240.py:768: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(models, rotation=30, ha='right', fontsize=8)
/tmp/ipython-input-1398/3455737240.py:768: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(models, rotation=30, ha='right', fontsize=8)
/tmp/ipython-input-1398/3455737240.py:768: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(models, rotation=30, ha='right', fontsize=8)


  [saved] ./outputs/trilemma_comparison.png
  [saved] ./outputs/diff_speed_quality_tradeoff.png

ALL SAVED FILES IN /content/outputs
  diff_label_dist_100steps.png                      41.2 KB
  diff_label_dist_200steps.png                      40.3 KB
  diff_label_dist_20steps.png                       41.5 KB
  diff_label_dist_500steps.png                      45.6 KB
  diff_label_dist_50steps.png                       40.9 KB
  diff_samples_100steps.png                        375.7 KB
  diff_samples_200steps.png                        361.4 KB
  diff_samples_20steps.png                         372.6 KB
  diff_samples_500steps.png                        174.7 KB
  diff_samples_50steps.png                         374.6 KB
  diff_speed_quality_tradeoff.png                   56.6 KB
  diff_training_curve.png                           34.1 KB
  gan_interpolation.png                             26.3 KB
  gan_label_dist.png                                39.9 KB
  gan_mode_collapse_watch.p

In [5]:
from google.colab import files
files.download('/content/outputs.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>